In [1]:
import pandas as pd 
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sub = pd.read_csv('gender_submission.csv', index_col=0)

In [2]:
train_x = train.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin']).copy()
train_y = train['Survived'].copy()

In [3]:
train_x['Sex'] = train_x['Sex'].astype('category').cat.codes
train_x['Embarked'] = train_x['Embarked'].astype('category').cat.codes

In [4]:
test_x = test.drop(columns=['Name', 'Ticket', 'Cabin']).copy()
test_x['Sex'] = test_x['Sex'].astype('category').cat.codes
test_x['Embarked'] = test_x['Embarked'].astype('category').cat.codes

In [5]:
from sklearn.model_selection import StratifiedKFold
random_state = 42
skf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)
idxs = []
for train_idx, valid_idx in skf.split(train_x, train_y):
    idxs.append([train_idx, valid_idx])

In [6]:
import lightgbm as lgb

In [7]:
import numpy as np 
test_ys = np.zeros([test_x.shape[0], 2])
for i in range(5):
    train_xi, train_yi = train_x.iloc[idxs[i][0]], train_y.iloc[idxs[i][0]]
    valid_xi, valid_yi = train_x.iloc[idxs[i][1]], train_y.iloc[idxs[i][1]]

    lgbmcls = lgb.LGBMClassifier(boosting_type='dart')
    lgbmcls.fit(train_xi, train_yi)
    valid_y_pred = lgbmcls.predict(valid_xi)
    print((valid_y_pred == valid_yi).sum() / valid_yi.shape[0])
    test_yi = lgbmcls.predict_proba(test_x)
    test_ys += test_yi

test_ys /= 5

0.8547486033519553
0.848314606741573
0.7865168539325843
0.8314606741573034
0.8426966292134831


In [8]:
test_out = []
for i in test_ys:
    test_out.append(0 if i[0] > i[1] else 1)

In [9]:
test_y = pd.Series(test_out, index=test_x.index, name='Survived')
test_y.to_csv('test_submission.csv')

In [10]:
saved_csv = pd.read_csv('test_submission.csv', index_col=0)
saved_csv.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
